In [ ]:
import os
import re
from openai import OpenAI
from tqdm import tqdm

# 模型接口地址和api_key
client = OpenAI(base_url="http://127.0.0.1:5000/v1", api_key="not-needed")

def translate_lrc_with_context(current_block, previous_block=""):
    """
    带着上一段的语境翻译当前这一段
    """
    system_prompt = "你是一位天才音乐翻译家，精通中英文语境，专门从事LRC歌词翻译，擅长捕捉歌词中的微妙情感，翻译风格优美、自然，严禁机械直译。"
    context_info = f"【前文语境参考】：\n{previous_block}\n" if previous_block else ""
    user_prompt = f"""任务：将以下英文LRC歌词翻译为简体中文。

{context_info}
【本次待翻译内容】：
{current_block}

规则：
1. 严格保留每行行首的时间戳，如 [00:12.34]，严禁修改或删除。
2. 必须保持【行对行】的对应关系，原文本有几行，翻译后必须也是几行。
3. 即使一句话被跨行切分，翻译后也要对应填入原行，确保按行对齐。
4. 意境要优美，符合歌词语境。
5. 仅输出翻译后的内容，不要任何解释。
"""
    
    try:
        response = client.chat.completions.create(
            model="local-model",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.4, # 稍微提高一点随机性，增加文学性
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"\n请求出错: {e}")
        return current_block

def process_lrc():
    input_dir = "./lists/"
    output_dir = "./output/"
    
    if not os.path.exists(input_dir):
        os.makedirs(input_dir)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 这些都过滤掉
    meta_keywords = ["歌名", "歌手", "作词", "作曲", "编曲", "演唱", "制作", "Artist", "Title", "Album"]

    # 获取所有 lrc 文件
    files = [f for f in os.listdir(input_dir) if f.endswith('.lrc')]
    
    for filename in tqdm(files, desc="翻译进度"):
        # 过滤空行
        with open(os.path.join(input_dir, filename), 'r', encoding='utf-8') as f:            
            lines = [line for line in f.read().splitlines() if line.strip()]

        processed_lines = []
        lyric_to_translate = []

        for line in lines:
            # 时间戳匹配
            match = re.match(r'^(\[.*\])(.*)', line)
            if match:
                timestamp, content = match.groups()
                # 检查内容是否包含关键字
                if any(kw in content for kw in meta_keywords):
                    processed_lines.append(f"{timestamp} ")
                else:
                    # 正常的歌词行，加入待翻译队列
                    lyric_to_translate.append(line)
            else:
                # 不带时间戳的原样保留（好像没看到这种）
                processed_lines.append(line)

        # 分批翻译歌词部分
        batch_size = 10
        translated_lyrics = []
        prev_batch = ""

        for i in range(0, len(lyric_to_translate), batch_size):
            current_batch = "\n".join(lyric_to_translate[i:i+batch_size])
            # 传入当前块和上一块作为参考
            translated_batch = translate_lrc_with_context(current_batch, prev_batch)
            translated_lyrics.append(translated_batch)
            # 更新上一块语境
            prev_batch = current_batch
            

        # 组合结果
        final_output = "\n".join(processed_lines + translated_lyrics)
        
        # 一行留空
        final_output = final_output.strip() + "\n"

        # 写入文件
        with open(os.path.join(output_dir, filename), 'w', encoding='utf-8') as f:
            f.write(final_output)

if __name__ == "__main__":
    process_lrc()
    print("\n所有任务已完成！请检查 ./output/ 文件夹。")


翻译进度: 100%|██████████| 2/2 [01:14<00:00, 37.21s/it]


所有任务已完成！请检查 ./output/ 文件夹。
